Questions
- do # of posts correlate with competitiveness of program?
- is there a specific day of the week admissions come out?
- distribution of GPA, Q,V, AWA
- Do rejections come out before acceptances?
- 



In [21]:
from bs4 import BeautifulSoup
import pandas as pd
from requests import get
import time
from random import randint
from IPython.core.display import clear_output
from warnings import warn
import altair as alt

In [80]:
berkeley_URL = "https://www.thegradcafe.com/survey/index.php?q=berkeley+Statistics&t=a&o=&pp=250"
washington_URL = "https://www.thegradcafe.com/survey/index.php?q=university+of+washington+data+science&t=a&o=&pp=250"
toronto_URL = "https://www.thegradcafe.com/survey/index.php?q=mscac&t=a&o=&p=1&pp=250"
northeastern_URL = "https://www.thegradcafe.com/survey/index.php?q=northeastern+data+science&t=a&o=&pp=250"
stanford_URL = "https://www.thegradcafe.com/survey/index.php?q=stanford+data+science&t=a&o=&pp=250"
harvard_URL = "https://www.thegradcafe.com/survey/index.php?q=harvard+university+data+science&t=a&o=&pp=250"
minnesota_URL = "https://www.thegradcafe.com/survey/index.php?q=Minnesota+business+analytics&t=a&o="
mit_URL="https://www.thegradcafe.com/survey/index.php?q=Mit+business+analytics&t=a&o=&pp=250"
cmu_URL = "https://www.thegradcafe.com/survey/index.php?q=cmu+bida&t=a&o=&pp=250"
waterloo_URL = "https://www.thegradcafe.com/survey/index.php?q=waterloo+data+science&t=a&o=&pp=250"
ubc_URL = "https://www.thegradcafe.com/survey/index.php?q=british+columbia+data+science&t=a&o=&pp=250"

In [116]:
colleges = ["Berkeley Statistics","Washington Data Science","Toronto MscAc","Northeastern Data Science","Stanford Data Science","Harvard Applied Computing","Minnesota Business Analytics","MIT Business Analytics","CMU Heinz BIDA","Waterloo MDSAI","UBC Data Science"]
college_names =[]
programs = []
statuses =[]
dates = []
stats =[]
comments =[]

start_time = time.time()
requests = 0
for url in [berkeley_URL,washington_URL, toronto_URL,northeastern_URL,stanford_URL,harvard_URL,minnesota_URL,mit_URL,cmu_URL,waterloo_URL,ubc_URL]:
    response = get(url)
    
    # Throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))
    
    time.sleep(randint(10,30))


    html_soup = BeautifulSoup(response.text, 'html.parser')
    
    #the website has alternate html classes for each row for some reason
    container1 = html_soup.find_all('tr', class_ = 'row0')
    container2 = html_soup.find_all('tr', class_ = 'row1')
    containers = container1 + container2
    
    

    
    # Monitor the requests
    
    elapsed_time = time.time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)


    for container in containers:
        
        if container.find('td'):
            #College Name
            college_names.append(colleges[requests])
                
            #Program
            if container.find("td",class_= "tcol2")!=None:
                program = container.find("td",class_= "tcol2").text
                programs.append(program)
            else:
                programs.append("NA")
                
            
            #status
            if (container.find("td",class_= "tcol3").strong!=None):
                status = (container.find("td",class_= "tcol3").strong.text)
                statuses.append(status)
            else:
                statuses.append("NA")
            
            
            #Date
            if container.find("td",class_= "tcol5")!=None:
                date = (container.find("td",class_= "tcol5").text)
                dates.append(date)
            else:
                dates.append("NA")

            #Stats
            if (container.find("td",class_ = "tcol3").a!=None):
                stat = (container.find("td",class_ = "tcol3").a.text)
                stats.append(stat)
            else:
                stats.append("NA")
                
            #Comments
            if (container.find("td",class_ = "tcol6")!=None):
                comment = (container.find("td",class_ = "tcol6").text)
                comments.append(comment)
            else:
                comments.append("NA")
                
    requests += 1    
        

Request:10; Frequency: 0.043771164254263285 requests/s


In [132]:
data = pd.DataFrame({'college': college_names,
'program': programs,
'status': statuses,
'date': dates,
'stats': stats,
'comments':comments
})

In [133]:
data["date"] = pd.to_datetime(data['date'])
data[["GPA","GRE"]] =data.stats.str.split(pat="GRE General",expand = True)
data["GPA"]=data.GPA.str.split("Undergrad GPA:",expand = True).iloc[:,1]
data[["Verbal","Quant","AWA"]] = data.GRE.str.split("/",expand = True).iloc[:,[2,3,4]]
data["Verbal"] =data["Verbal"].str.split(":",expand = True).iloc[:,1]
data["AWA"] =data["AWA"].str.split("GRE",expand = True).iloc[:,0]

In [134]:
data = data.drop(["stats","GRE"],axis = 1)

In [135]:
data = data[data["program"].str.contains("Masters")]

In [136]:
data = data[pd.DatetimeIndex(data["date"]).year>2015]

In [140]:
alt.Chart(data).mark_rect(opacity = 0.4).encode(
    x = "monthdate(date)",
    y = "year(date)",
    color = "status",
    tooltip = ["status","date","comments","GPA","Verbal","Quant","AWA"]
    
).interactive().configure_axisY(grid = True)

alt.Chart(...)

In [37]:
berkeley[pd.DatetimeIndex(berkeley["date"]).year ==2019]

,college,program,status,date,comments,GPA,Verbal,Quant,AWA
10,"University Of California, Berkeley","Statistics, Masters (F19)",Rejected,2019-03-27,😋,None,None,None,None
12,UC Berkeley,"Statistics, Masters (F19)",Accepted,2019-03-25,,3.90,160,170,5.00
22,"University Of California, Berkeley","Statistics, Masters (F19)",Accepted,2019-04-09,,3.10,162,152,4.00
23,"University Of California, Berkeley","Applied Statistics, Masters (F19)",Rejected,2019-03-26,Beautiful life.,None,None,None,None
24,"University Of California, Berkeley","Statistics, Masters (F19)",Rejected,2019-03-26,"6th rejections, my life is so hard...",3.90,160,169,4.00
25,"University Of California, Berkeley","Statistics, Masters (F19)",Accepted,2019-03-18,,3.50,162,166,6.00
26,"University Of California, Berkeley","Statistics, Masters (F19)",Accepted,2019-03-06,,None,None,None,None
27,"University Of California, Berkeley","Statistics, Masters (F19)",Accepted,2019-03-05,SO THRILLED,n/a,168,163,0.00
28,"University Of California, Berkeley","Statistics, Masters (F19)",Accepted,2019-03-05,,3.93,164,170,4.00
39,"University Of California, Berkeley","Statistics, Masters (F19)",Accepted,2019-03-05,,3.92,152,170,3.50
